### N Gram Analysis According To Fourgram

This notebook is used to analyze the combined or separate use of 100 selected four grams and two and three grams.

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [4]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df,column_list):
    '''
    strip(df,column_list) df is dataframe and column_list are selected columns \n
    strip(df_twogram_used,df_twogram_used.columns)
    '''
    for i in column_list:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [9]:
def df_fill_nan_value(text_value,used_list):
    '''df_fill_nan_value(text_value,used_list) text_value from df column text value\n
    if text_value in used_list text list, func fill nan value else return same value\n
    ex.\n
    df_ngram_in_all_result["twogram"].apply(lambda x: df_fill_nan_value(x,twogram_used_list))
    '''
    if text_value in used_list:
        return np.nan
    else:
        return text_value

#### Ngram Read

In [10]:
df_twogram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.xlsx")  # used in previous 200 word level
df_twogram_used = strip(df_twogram_used,df_twogram_used.columns)
df_twogram_used.drop_duplicates(inplace=True)
df_twogram_used.reset_index(drop=True, inplace=True)
df_twogram_used

,twogram
0,bir şey
1,bu kadar
2,ne kadar
3,ve de
4,ne için
...,...
152,senin gibi
153,sorun değil
154,sorun yok
155,tamam mı


In [11]:
df_threegram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Used.xlsx")  # used in previous 200 word level
df_threegram_used = strip(df_threegram_used, df_threegram_used.columns)
df_threegram_used.drop_duplicates(inplace=True)
df_threegram_used.reset_index(drop=True, inplace=True)
df_threegram_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
333,bir daha hiç
334,bir de şu
335,bir daha asla
336,peki ne için


In [12]:
twogram_used_list = list(set(df_twogram_used["twogram"]))
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [13]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
220163,dışarıda bizimle,6
220164,bizim kimseye,6
220165,tarafa bunu,6
220166,beri kahrolası,6


In [14]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
500343,sonra insanlar bu,10
500344,o anda dünya,10
500345,zaman zaman sizinle,10
500346,bir güç sahibi,10


In [15]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


In [16]:
twogram_list = df_twogram["twogram"].to_list()
threegram_list = df_threegram["threegram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

#### Give Reveal Course

##### Twogram In Threegram In Selected Fourgram (1B)

In [17]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [18]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_in_selected_four

,threegram,fourgram
0,bir şey yok,diye bir şey yok
1,bir şey yok,başka bir şey yok
2,bir şey yok,bir şey yok mu
3,bir şey yok,hiç bir şey yok
4,bir şey yok,öyle bir şey yok
...,...,...
195,sakin ol tamam,sakin ol tamam mı
196,istiyor musun istemiyor,istiyor musun istemiyor musun
197,şey söyleyeyim mi,bir şey söyleyeyim mi
198,olur ne olmaz,ne olur ne olmaz


In [19]:
df_threegram_in_selected_four["fourgram"].nunique()

100

In [20]:
df_threegram_in_selected_four2 = df_threegram_in_selected_four[["threegram"]]
df_threegram_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_in_selected_four2 = df_threegram_in_selected_four2.reset_index(drop=True)
df_threegram_in_selected_four2

<ipython-input-20-d1ef5e5fe86d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threegram_in_selected_four2.drop_duplicates(inplace=True)


,threegram
0,bir şey yok
1,bir şey var
2,bu da ne
3,her şey yolunda
4,başka bir şey
...,...
158,sakin ol tamam
159,istiyor musun istemiyor
160,şey söyleyeyim mi
161,olur ne olmaz


In [21]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [22]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_in_threegram_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_in_threegram_in_selected_four2

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,başka bir şey
3,bir şey,bir şey değil
4,bir şey,bir şey mi
...,...,...
321,şey buldun,bir şey buldun
322,şey buldun,şey buldun mu
323,öyle ya,öyle ya da
324,şey kontrol,her şey kontrol


In [23]:
df_twogram_in_three_in_selected_four = pd.merge(df_twogram_in_threegram_in_selected_four2, df_threegram_in_selected_four, how="inner", on="threegram")
df_twogram_in_three_in_selected_four.drop_duplicates(inplace=True)
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,bir şey,bir şey yok,diye bir şey yok
1,bir şey,bir şey yok,başka bir şey yok
2,bir şey,bir şey yok,bir şey yok mu
3,bir şey,bir şey yok,hiç bir şey yok
4,bir şey,bir şey yok,öyle bir şey yok
...,...,...,...
395,şey kontrol,şey kontrol altında,her şey kontrol altında
396,ihtiyacın olursa,şeye ihtiyacın olursa,bir şeye ihtiyacın olursa
397,şeye ihtiyacın,şeye ihtiyacın olursa,bir şeye ihtiyacın olursa
398,ne düşünüyorum,ne düşünüyorum biliyor,ne düşünüyorum biliyor musun


In [24]:
df_twogram_in_three_in_selected_four = df_twogram_in_three_in_selected_four[["fourgram","threegram","twogram"]]
df_twogram_in_three_in_selected_four

,fourgram,threegram,twogram
0,diye bir şey yok,bir şey yok,bir şey
1,başka bir şey yok,bir şey yok,bir şey
2,bir şey yok mu,bir şey yok,bir şey
3,hiç bir şey yok,bir şey yok,bir şey
4,öyle bir şey yok,bir şey yok,bir şey
...,...,...,...
395,her şey kontrol altında,şey kontrol altında,şey kontrol
396,bir şeye ihtiyacın olursa,şeye ihtiyacın olursa,ihtiyacın olursa
397,bir şeye ihtiyacın olursa,şeye ihtiyacın olursa,şeye ihtiyacın
398,ne düşünüyorum biliyor musun,ne düşünüyorum biliyor,ne düşünüyorum


In [25]:
df_twogram_in_three_in_selected_four["fourgram"].nunique()

100

In [26]:
#df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx", index=False)  # 1B

##### Twogram In Selected Fourgram (2B)

In [27]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [28]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
295,mi hayır,evet mi hayır mı
296,yapıyorsun değil,şaka yapıyorsun değil mi
297,şey buldun,bir şey buldun mu
298,öyle ya,öyle ya da böyle


In [29]:
#df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_two_in_selected_four_list

In [30]:
df_two_in_selected_four2 = df_two_in_selected_four[~df_two_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_two_in_selected_four2.reset_index(drop=True, inplace=True)
df_two_in_selected_four2

,twogram,fourgram


In [31]:
df_two_in_selected_four2["fourgram"].nunique()

0

In [32]:
#df_two_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx", index=False)  # 2B

##### Threegram In Selected Fourgram (4B)

In [33]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [34]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,bir şey yok,diye bir şey yok
1,bir şey yok,başka bir şey yok
2,bir şey yok,bir şey yok mu
3,bir şey yok,hiç bir şey yok
4,bir şey yok,öyle bir şey yok
...,...,...
195,sakin ol tamam,sakin ol tamam mı
196,istiyor musun istemiyor,istiyor musun istemiyor musun
197,şey söyleyeyim mi,bir şey söyleyeyim mi
198,olur ne olmaz,ne olur ne olmaz


In [35]:
#df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_three_in_selected_four_list

In [36]:
df_three_in_selected_four2 = df_three_in_selected_four[~df_three_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_three_in_selected_four2.reset_index(drop=True, inplace=True)
df_three_in_selected_four2

,threegram,fourgram


In [37]:
#df_three_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx", index=False)  # 4B

##### Twogram In Threegram (3B)

In [37]:
set_two_1B_list = list(set(df_twogram_in_three_in_selected_four["twogram"]))
set_two_2B_list = list(set(df_two_in_selected_four2["twogram"]))
set_two_list = set_two_1B_list + set_two_2B_list
#set_two_list

In [38]:
df_two_remove_used = df_twogram[~df_twogram["twogram"].isin(set_two_list)]
df_two_remove_used

,twogram,frequency
4,ne oldu,322758
6,hayır hayır,266457
8,öyle mi,263184
9,özür dilerim,260661
11,ne yapıyorsun,236176
...,...,...
220163,dışarıda bizimle,6
220164,bizim kimseye,6
220165,tarafa bunu,6
220166,beri kahrolası,6


In [39]:
set_three_1B_list = list(set(df_twogram_in_three_in_selected_four["threegram"]))
set_three_4B_list = list(set(df_three_in_selected_four2["threegram"]))
set_three_list = set_three_1B_list+set_three_4B_list
#set_three_list

In [40]:
df_three_remove_used = df_threegram[~df_threegram["threegram"].isin(set_three_list)]
df_three_remove_used

,threegram,frequency
5,hayır hayır hayır,69443
7,öyle değil mi,68442
9,ne demek istiyorsun,63142
13,sen iyi misin,42180
19,için özür dilerim,36255
...,...,...
500343,sonra insanlar bu,10
500344,o anda dünya,10
500345,zaman zaman sizinle,10
500346,bir güç sahibi,10


In [41]:
df_two_remove_used_list = list(set(df_two_remove_used["twogram"]))

In [42]:
# parameter
source_wordgroup_list = df_two_remove_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_remove_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [43]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three_not_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three_not_in_selected_four

,twogram,threegram
0,ihtiyacımız olmayan,ihtiyacımız olmayan bir
1,ihtiyacımız olmayan,artık ihtiyacımız olmayan
2,ihtiyacımız olmayan,ve ihtiyacımız olmayan
3,olmayacak bile,hiç olmayacak bile
4,kolay olmaz,daha kolay olmaz
...,...,...
954736,buna mı,herşey buna mı
954737,buna mı,buna mı göz
954738,buna mı,siz buna mı
954739,buna mı,yine buna mı


In [44]:
df_two_in_three_not_in_selected_four_freq = pd.merge(df_two_in_three_not_in_selected_four, df_threegram, how="left", on="threegram")
df_two_in_three_not_in_selected_four_freq.drop_duplicates(inplace=True)
df_two_in_three_not_in_selected_four_freq.sort_values(by="frequency", ascending=False, inplace=True)
df_two_in_three_not_in_selected_four_freq_select = df_two_in_three_not_in_selected_four_freq.head(100)
df_two_in_three_not_in_selected_four = df_two_in_three_not_in_selected_four_freq_select.drop("frequency", axis=1)
df_two_in_three_not_in_selected_four

,twogram,threegram
300817,hayır hayır,hayır hayır hayır
863860,öyle değil,öyle değil mi
780366,demek istiyorsun,ne demek istiyorsun
420722,iyi misin,sen iyi misin
659680,sen iyi,sen iyi misin
...,...,...
461451,çok kötü,çok kötü bir
297297,şey olmaz,bir şey olmaz
642073,bir bak,şuna bir bak
456935,şuna bir,şuna bir bak


In [48]:
df_two_in_three_not_in_selected_four_freq

,twogram,threegram,frequency
300817,hayır hayır,hayır hayır hayır,69443
863860,öyle değil,öyle değil mi,68442
780366,demek istiyorsun,ne demek istiyorsun,63142
420722,iyi misin,sen iyi misin,42180
659680,sen iyi,sen iyi misin,42180
...,...,...,...
43188,siz de,siz de değil,10
658255,ki hep,ki hep yanlış,10
349033,kardeşim bir,kardeşim bir daha,10
349032,kardeşim bir,kardeşim bir içki,10


In [47]:
#df_two_in_three_not_in_selected_four_freq_select.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx", index=False)
#df_two_in_three_not_in_selected_four_freq.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_All_Result.xlsx", index=False)

##### All Result Concat

In [ ]:
df_fourgram_selected_for_course

In [81]:
df_ngram_in_result = pd.concat([df_twogram_in_three_in_selected_four,df_three_in_selected_four2,df_two_in_selected_four2,df_two_in_three_not_in_selected_four], axis=0)
df_ngram_in_result.reset_index(drop=True, inplace=True)
df_ngram_in_result

,fourgram,threegram,twogram
0,diye bir şey yok,bir şey yok,bir şey
1,başka bir şey yok,bir şey yok,bir şey
2,bir şey yok mu,bir şey yok,bir şey
3,hiç bir şey yok,bir şey yok,bir şey
4,öyle bir şey yok,bir şey yok,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,şey olmaz
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,şuna bir


In [82]:
df_ngram_in_result["fourgram"].nunique()

100

In [83]:
ngram_in_four_list = list(set(df_ngram_in_result["fourgram"]))

In [84]:
df_four_not_pair = df_fourgram_selected_for_course[~df_fourgram_selected_for_course["fourgram"].isin(ngram_in_four_list)]
df_four_not_pair = df_four_not_pair.drop("frequency", axis=1)
df_four_not_pair

,fourgram


In [85]:
df_ngram_in_all_result = pd.concat([df_ngram_in_result,df_four_not_pair], axis=0)
df_ngram_in_all_result.reset_index(drop=True, inplace=True)
df_ngram_in_all_result

,fourgram,threegram,twogram
0,diye bir şey yok,bir şey yok,bir şey
1,başka bir şey yok,bir şey yok,bir şey
2,bir şey yok mu,bir şey yok,bir şey
3,hiç bir şey yok,bir şey yok,bir şey
4,öyle bir şey yok,bir şey yok,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,şey olmaz
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,şuna bir


In [86]:
df_ngram_in_all_result["fourgram"].nunique()

100

In [88]:
df_ngram_in_all_result_freq = pd.merge(df_ngram_in_all_result, df_fourgram_selected_for_course, how="left", on="fourgram")
df_ngram_in_all_result_freq = df_ngram_in_all_result_freq.sort_values(by="frequency",ascending=False)
df_ngram_in_all_result_freq

,fourgram,threegram,twogram,frequency
19,bir şey var mı,bir şey var,şey var,41773.0
14,bir şey var mı,bir şey var,bir şey,41773.0
126,bir şey var mı,şey var mı,var mı,41773.0
127,bir şey var mı,şey var mı,şey var,41773.0
151,her şey yolunda mı,her şey yolunda,şey yolunda,31126.0
...,...,...,...,...
495,NaN,çok kötü bir,çok kötü,NaN
496,NaN,bir şey olmaz,şey olmaz,NaN
497,NaN,şuna bir bak,bir bak,NaN
498,NaN,şuna bir bak,şuna bir,NaN


In [55]:
df_ngram_in_all_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Threegram_Twogram_In_Each_Other.xlsx", index=False)

In [89]:
df_ngram_in_all_result_freq.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Threegram_Twogram_In_Each_Other_Freq.xlsx", index=False)

##### Remove Used N Gram 

In [61]:
df_ngram_in_all_result_remove_used_freq = df_ngram_in_all_result_freq

In [91]:
df_ngram_in_all_result_remove_used_freq["twogram"] = df_ngram_in_all_result_freq["twogram"].apply(lambda x: df_fill_nan_value(x,twogram_used_list))

In [93]:
df_ngram_in_all_result_remove_used_freq["threegram"] = df_ngram_in_all_result_freq["threegram"].apply(lambda x: df_fill_nan_value(x,threegram_used_list))

In [96]:
df_ngram_in_all_result_remove_used_freq.drop_duplicates(inplace=True)
df_ngram_in_all_result_remove_used_freq

In [98]:
df_ngram_in_all_result_remove_used_freq.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Threegram_Twogram_In_Each_Other_Without_Prev_Used_Freq.xlsx", index=False)

#### Copy Move And Delete

In [90]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Threegram_Twogram_In_Each_Other*.xlsx")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Twogram_Alone_(5)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_(1A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Selected_Fourgram_(2A)_Result.xlsx',
 'Turkish_Threegram_Used_In_Selected_Fourgram_(4A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_Not_In_Selected_Fourgram_(3A)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx',
 'Turkish_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx',
 'Turkish_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx',
 'Turkish_Threegram_Alone_(6)_Result.xlsx',
 'Turkish_Fourgram_Alone_(7)_Result.xlsx']

In [91]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [92]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass